
## CLASSIFICATION - AUTHENTIFICATION DES BILLETS DE BANQUE 


Notre tutoriel porte sur l'authentification de billets de banque. Pour cela nous allons utiliser le dataset suivant: https://www.kaggle.com/datasets/ritesaluja/bank-note-authentication-uci-data (disponible sur Kaggle).    

Il permet de créer un modèle capable de détecter les billets de banque authentiques et faux. Ce jeu de données contient un certain nombre de mesures prises à partir d'images numérisées. Les images sont créées à l'aide d'une caméra industrielle qui est habituellement utilisée pour l'inspection des imprimés. Les images sont de 400x 400 pixels. 

Dans ce notebook nous allons nous concentrer sur un exemple de classification binaire, où le modèle doit prédire si un billet de banque est une contrefaçon ou pas. 

## Exploration des données 

In [1]:
#Télecharger au préalable le fichier à travers le lien, dans le même dossier que ce notebook.

#Charger le fichier csv de données 
import pandas as pd
Bank_data = pd.read_csv("BankNote_Authentication.csv")
Bank_data.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


## Description des données

Ce jeu de données contient 5 colonnes: une étiquette "class" qui identifie le statut d'un billet de banque (0 pour faux et 1 pour vrai) et quatres caractéristiques. 

Les quatre caractéristiques ont été extraites des images de billets de banque:

-"Variance": mesure l'étendue de la dispersion d'un ensemble de nombres.

-"Skewness": mesure l'asymétrie de la distribution de probabilité d'une variable aléatoire à valeur réelle.

-"Curtosis": mesure la forme de la distribution de probabilité d'une variable aléatoire à valeur réelle.

-"Entropy": mesure la quantité d'information ou du caractère aléatoire, qui est représentée par la différence entre des pixels adjacents.

Il s'agit d'un excellent jeu de données pour pratiquer la classification binaire.


## Nettoyage des données

Après avoir légèrement exploré nos données, nous pouvons passer au nettoyage de celles-ci. Le fait de disposer de données propres augmentera en fin de compte la productivité globale et permettra de disposer d'informations de la plus haute qualité pour entrainer le modèle.

Les données incomplètes ou manquantes constituent l'un des problèmes les plus courants auxquels les spécialistes des données doivent faire face. Alors comment savoir si le DataFrame contient des valeurs manquantes ? On peut utiliser les fonctions isnull() et sum() pour identifier les valeurs qui sont nulles, comme ceci :

In [2]:
Bank_data.isnull().sum()

variance    0
skewness    0
curtosis    0
entropy     0
class       0
dtype: int64

Aucune valeur nulle. Nous remarquons aussi que nous n'avons pas de problèmes de typage toutes les caractéristiques sont des décimaux relatifs et la classe un entier ce qui colle à la réalité. 

In [3]:
Bank_data.dtypes

variance    float64
skewness    float64
curtosis    float64
entropy     float64
class         int64
dtype: object

Nous remarquons que les différentes caractéristiques ont des échelles différentes ce qui peut fausser les calculs. Une technique courante pour traiter des données numériques à différentes échelles consiste à normaliser les données afin que les valeurs conservent leur distribution proportionnelle, mais soient mesurées sur la même échelle. Pour ce faire, nous utiliserons une technique appelée mise à l'échelle MinMax qui distribue les valeurs proportionnellement sur une échelle de 0 à 1. Vous pourriez écrire le code pour appliquer cette transformation, mais la bibliothèque Scikit-Learn fournit un mesureur qui le fait pour vous.

In [4]:
from sklearn.preprocessing import MinMaxScaler

# Obtenir un objet de mesure
scaler = MinMaxScaler()

# Créer un nouveau dataframe pour les valeurs mises à l'échelle.
Bank_data_normalized = Bank_data[['variance', 'skewness', 'curtosis', 'entropy', 'class']].copy()

# Normaliser les colonnes numériques
Bank_data_normalized[['variance', 'skewness', 'curtosis', 'entropy']] = scaler.fit_transform(Bank_data_normalized[['variance', 'skewness', 'curtosis', 'entropy']])

Bank_data_normalized

,variance,skewness,curtosis,entropy,class
0,0.769004,0.839643,0.106783,0.736628,0
1,0.835659,0.820982,0.121804,0.644326,0
2,0.786629,0.416648,0.310608,0.786951,0
3,0.757105,0.871699,0.054921,0.450440,0
4,0.531578,0.348662,0.424662,0.687362,0
...,...,...,...,...,...
1367,0.537124,0.565855,0.165249,0.726398,1
1368,0.407690,0.332868,0.506753,0.808350,1
1369,0.237385,0.011768,0.985603,0.524755,1
1370,0.250842,0.201701,0.761587,0.660675,1


## Séparer les étiquettes et les caractéristiques

L'étiquette sera prédite par le modèle entrainé; les caractéristiques seront utilisées pour prédire le statut du billet. 

Il est nécéssaire de séparer au préalable les caractéristiques des étiquettes - nous appellerons les caractéristiques X et l'étiquette Y :

In [5]:
#on supprime class pour avoir les entrées(les caractéristiques).
X = Bank_data_normalized.drop(columns=['class'])

#on supprime les caractéristiques pour avoir la sortie(étiquette).
y = Bank_data_normalized.drop(columns=['variance','skewness','curtosis','entropy'])


## Diviser les données

Nous pouvons tirer parti du fait que nous disposons d'un grand ensemble de données avec des valeurs d'étiquettes connues, pour n'en utiliser qu'une partie pour entrainerle modèle, et en retenir une autre  pour tester le modèle formé - ce qui nous permet de comparer les étiquettes prédites avec les étiquettes déjà connues dans l'ensemble du test.

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


## Former un modèle de classification binaire

In [7]:
#Création du modèle d'entrainement
model = DecisionTreeClassifier()

#Entrainement du modèle
model.fit(X_train.values, y_train)

#Prédiction des données
predictions = model.predict (X_test.values)

## Evaluer le modèle

Maintenant que nous avons formé le modèle à l'aide des données d'entrainement, nous pouvons utiliser les données de test que nous avons retenues pour évaluer la qualité de ses prédictions. La chose la plus évidente à faire est de vérifier la précision des prédictions - en termes simples, quelle proportion des étiquettes le modèle a-t-il prédit correctement ? Pour cela nous pouvons nous servir de l'accuracy score. Il s'agit d'une métrique permettant d'évaluer la performance des modèles de classification à 2 classes ou plus. 

In [8]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, predictions)
score 

0.9854368932038835

Maintenant que nous disposons d'un modèle entraîné raisonnablement utile, nous pouvons l'enregistrer pour l'utiliser ultérieurement afin de prédire les étiquettes de nouvelles données :

In [9]:
 import joblib

joblib.dump(model, 'Bank-model.joblib')

['Bank-model.joblib']

## Réutilisation du modèle

Lorsque nous avons de nouvelles observations pour lesquelles l'étiquette est inconnue, nous pouvons charger le modèle et l'utiliser pour prédire les valeurs de l'étiquette inconnue :

In [10]:
import numpy as np
# Charger le modèle grâce au fichier sauvegardé
model = joblib.load('Bank-model.joblib')

#Nous allons créer un tableau avec un seul tableau de caractéristiques, représentant un billet.
X_new = np.array([[2,3,4,1]])
print ('Nouvel élément: {}'.format(list(X_new[0])))

# Récupérer la prédiction
pred = model.predict(X_new)

# Le modèle renvoie un tableau de prédictions - une pour chaque ensemble de caractéristiques soumises.
# Dans notre cas, nous n'avons soumis qu'un seul élément, donc notre prédiction est la première dans le tableau résultant.
print('la classe prédite est {}'.format(pred[0]))

Nouvel élément: [2, 3, 4, 1]
la classe prédite est 0
